In [2]:
#import ale_py
#import shimmy
import gymnasium as gym
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, Dataset
from torch.nn import functional as F
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
#import os
import random
import copy
import time
import pickle
#import torcheck
import torchvision
import matplotlib.pyplot as plt


In [3]:

class CNN(nn.Module):

    def __init__(self):
        super().__init__()   
        
        self.stack = torch.nn.Sequential(
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 2)
        )

    def forward(self, x):
        x = self.stack(x)
        return x


In [28]:
class DQN_agent:

    def __init__(self):
        # Get cpu, gpu or mps device for training.
        self.device = (
            "cuda"
            if torch.cuda.is_available()
            else "mps"
            if torch.backends.mps.is_available()
            else "cpu"
        )
        print(f"Using {self.device} device")
        self.pred_NN = CNN().to(self.device)
        self.target_NN = copy.deepcopy(self.pred_NN)
    
    def predict(self, x):
        pred = self.pred_NN.forward(x)
        return pred
    
    def action(self, pred):
        if random.random() < epsilon:
            action = random.randint(0, pred.size(dim=0) - 1)
        else: 
            action = torch.argmax(pred).item()
        return action
    
    def train(self, experiences):
      
        data_set = ExperiencesDataset(experiences, self)
        data_loader = DataLoader(data_set, batch_size=samples_per_batch, shuffle=True)
        optimizer = torch.optim.SGD(self.pred_NN.parameters(), lr=0.1)
        loss_fn = nn.MSELoss()
        epoch_loss = 0
        
        self.pred_NN.train()

        #torcheck.register(optimizer)
        #torcheck.add_module_changing_check(self.pred_NN, module_name="my_model")
        
        for batch, (state, y) in enumerate(data_loader):
            
            pred = self.pred_NN(state)
            loss = loss_fn(pred, y)
            
            loss.backward(retain_graph=True)
            #print('****** Gradients ******')
            #for p in self.pred_NN.parameters():
                #print(p.grad.norm())
            optimizer.step()
            epoch_loss += loss.item()
            optimizer.zero_grad()
            
            if batch == (len(data_loader) - 1):
                self.pred_NN.train(mode=False)
                return epoch_loss
        
    def copy(self):
        self.target_NN = copy.deepcopy(self.pred_NN)
        

In [29]:
class ExperiencesDataset(Dataset):
    
    def __init__(self, experiences, agent):
    
        self.states = []
        self.labels = []
        
        for n, experience in enumerate(experiences):
            #print('**************  CREATE LABEL  ********************\n')
            state, action, reward, next_state = experience
            
            #print('STATE\n')
            #print(state, '\n')
            #img = state[0].permute(1, 2, 0).numpy()
            #plt.imshow(img)
            #plt.show()
            
            #print('NEXT STATE\n')
            #print(next_state, '\n')
            #img = next_state[0].permute(1, 2, 0).numpy()
            #plt.imshow(img)
            #plt.show()

            #print('reward: ', reward, '\n')
            #print('action: ', action, '\n')
            
            pred = agent.predict(state)
            #label = pred.detach().clone().squeeze()
            label = pred.detach().clone()
            
            #print('pred: ', label, '\n')
            
            target_pred = agent.target_NN.forward(next_state)
            #target_pred = target_pred.squeeze()
            target = reward + gamma * max(target_pred)
            
            
            label[action] = target
            #print('label: ', label, '\n')
            
            self.labels.append(label) 
            
            #state = state.squeeze()
            self.states.append(state)
            #print('********************  END  ***********************\n')
    def __len__(self):
        return len(self.states)
    
    def __getitem__(self, idx):
        return self.states[idx], self.labels[idx]

In [30]:
#initialize environment
env = gym.make("CartPole-v1",render_mode="rgb_array")
#preprocess environment
#env = gym.wrappers.AtariPreprocessing(env, screen_size=84, grayscale_obs=True, frame_skip=1, noop_max=30, scale_obs = True)
#start environment
state, info = env.reset()
#create action array
actions = range(env.action_space.n)

#hyperparams
epsilon = 0.2
epsilon_decay = 1
gamma = 0.75
experience_capacity = 1200
training_freq = 100
copying_freq = 400
samples_per_training_session = 100
samples_per_batch = 20

#initialize agent
agent = DQN_agent()
#agent.pred_NN.train(mode=False)
#agent.target_NN.train(mode=False)

#load saved model to agent NNs
#agent.pred_NN.load_state_dict(torch.load("model"))
#agent.target_NN = copy.deepcopy(agent.pred_NN)

#select first action
action = random.choice(actions)
print('action: ', action)

#get first states
state1, reward, terminated, truncated, info = env.step(action)
state1 = torch.from_numpy(state1)
#state2, reward, terminated, truncated, info = env.step(action)
#state3, reward, terminated, truncated, info = env.step(action)
#state4, reward, terminated, truncated, info = env.step(action)

#convert first states to correct datatype and stack
#state1 = TF.to_tensor(state1)     
#state2 = TF.to_tensor(state2)
#state3 = TF.to_tensor(state3)
#state4 = TF.to_tensor(state4)
#state_stack = torch.cat((state1,state2,state3,state4))
#state_stack = torch.unsqueeze(state_stack, 0)

#plot image, to se resolution
#img = state4.permute(1, 2, 0).numpy()
#plt.imshow(img)
#plt.show()

print('Plot filters in first layer\n')
#kernels = agent.pred_NN.stack[0].weight.detach().clone()
#kernels = kernels - kernels.min()
#kernels = kernels / kernels.max()
#filter_img = torchvision.utils.make_grid(kernels, nrow = 8)
#plt.imshow(filter_img.permute(1, 2, 0))
#plt.show() 

#timer and variables connected to time
start_time = time.time()
epsilon_start_time = time.time()
elapsed_time = 0
max_time = 720
training_time = 0
decay_freq = max_time/16

#for logging
training_session = 0


#loops until max_time is reached
while elapsed_time < max_time:
    experiences = []
    #print(f'*********  Elapsed time: {elapsed_time} seconds  *************\n')
    #loops until experience_capacity is reached
    for i in range(1, experience_capacity):
        
    
        #predict q-values and choose action
        pred = agent.predict(state1)
        #pred = pred.squeeze()
        action = agent.action(pred)
        #if action == 0:
            #print('state: ', state1 , '\n', 'action: ', 'left')
        #if action == 1:
            #print('state: ', state1 , '\n', 'action: ', 'right')
        
        #get next states
        next_state1, reward1, terminated, truncated, info = env.step(action)
        #next_state2, reward2, terminated, truncated, info = env.step(action)
        #next_state3, reward3, terminated, truncated, info = env.step(action)
        #next_state4, reward4, terminated, truncated, info = env.step(action)
        states_alive += 1
        #if pac-man dies 4 times, reset and start again
        if terminated or truncated:
            states_alive_list.append(states_alive)
            states_alive = 0
            #print('terminated:', terminated, 'truncated: ', truncated)
            #restart pacman
            env.reset()
            #select first action
            action = random.choice(actions)
            #get first states
            next_state1, reward1, terminated, truncated, info = env.step(action)
            #next_state2, reward2, terminated, truncated, info = env.step(action)
            #next_state3, reward3, terminated, truncated, info = env.step(action)
            #next_state4, reward4, terminated, truncated, info = env.step(action)
            
        #convert next state to correct type and stack
        #next_state1 = TF.to_tensor(next_state1)
        next_state1 = torch.from_numpy(next_state1)
        #next_state2 = TF.to_tensor(next_state2)
        #next_state3 = TF.to_tensor(next_state3)
        #next_state4 = TF.to_tensor(next_state4)
        #next_state_stack = torch.cat((next_state1,next_state2,next_state3,next_state4)) 
        #next_state_stack = torch.unsqueeze(next_state_stack, 0)
        
        #sum rewards 
        #tot_reward = 0
        tot_reward = reward1 #+ reward2 + reward3 + reward4
        
        #add to experiences
        save_this = [state1, action, tot_reward, next_state1] 
        experiences.append(save_this)
        
        #train
        if i % training_freq == 0:
            training_session += 1
            #print(f'-----Training {training_session}-----')
            elapsed_time = time.time() - start_time
            #print(f'Elapsed time: {elapsed_time} seconds')
            experiences_train = random.choices(experiences, k=samples_per_training_session)
            epoch_loss = agent.train(experiences_train)
            #print("Epoch loss: ", epoch_loss, "\n")  
            
        #copy
        if i % copying_freq == 0:
            #print('-----Copying pred_NN to target_NN-----\n')
            agent.copy()
            elapsed_time = time.time() - start_time
            if elapsed_time > max_time:
                break
                
        #next state stack becomes current state stack
        #state_stack = next_state_stack
        state1 = next_state1
            
        
    #decay epsilon
    #if(epsilon > 0.2 and (time.time() - epsilon_start_time) > decay_freq):
    if True:
        elapsed_time = time.time() - start_time
        print(f'*********  Elapsed time: {elapsed_time} seconds  *************\n')
        print(f'------Epsilon decays after {(time.time() - epsilon_start_time)} seconds since last decay-----\n')
        print(f'old epsilon: {epsilon} \n')
        epsilon = epsilon * epsilon_decay 
        print(f'new epsilon: {epsilon}\n')
        average_states_alive = sum(states_alive_list)/len(states_alive_list)
        print('Average states alive: ', average_states_alive, '\n')
        states_alive = 0
        states_alive_list = []
        epsilon_start_time = time.time()
        

print(f'*****stopped training after {elapsed_time} seconds*****\n')

print('*****Plot filters in first layer*****\n')
kernels = agent.pred_NN.stack[0].weight.detach().clone()
kernels = kernels - kernels.min()
kernels = kernels / kernels.max()
filter_img = torchvision.utils.make_grid(kernels, nrow = 8)
plt.imshow(filter_img.permute(1, 2, 0))
plt.show() 

print('*****Pickle Dumping model*****\n')

file = open('safetyPickleDump', 'wb')
pickle.dump(agent, file)
file.close()

print("*****Saving model*****")
torch.save(agent.pred_NN.state_dict(), "model")

env.close()

Using cpu device
action:  1
Plot filters in first layer

*********  Elapsed time: 0.7742338180541992 seconds  *************

------Epsilon decays after 0.7742338180541992 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  12.808 

*********  Elapsed time: 1.543492317199707 seconds  *************

------Epsilon decays after 0.7692584991455078 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.625 

*********  Elapsed time: 2.3118021488189697 seconds  *************

------Epsilon decays after 0.7693228721618652 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.575221238938052 

*********  Elapsed time: 3.0628387928009033 seconds  *************

------Epsilon decays after 0.750023603439331 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.575221238938052 

*********  Elapsed time: 3.8206803798675537 seconds  ************

*********  Elapsed time: 35.30588746070862 seconds  *************

------Epsilon decays after 0.9029455184936523 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.836363636363636 

*********  Elapsed time: 36.19988775253296 seconds  *************

------Epsilon decays after 0.8940002918243408 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.845454545454546 

*********  Elapsed time: 37.09685516357422 seconds  *************

------Epsilon decays after 0.8969674110412598 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  11.064814814814815 

*********  Elapsed time: 38.00523400306702 seconds  *************

------Epsilon decays after 0.9083788394927979 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.6875 

*********  Elapsed time: 38.90297055244446 seconds  *************

------Epsilon decays after 0.897736549377441

*********  Elapsed time: 68.89071536064148 seconds  *************

------Epsilon decays after 0.8933734893798828 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.5 

*********  Elapsed time: 69.77884006500244 seconds  *************

------Epsilon decays after 0.8881247043609619 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.347826086956522 

*********  Elapsed time: 70.67681121826172 seconds  *************

------Epsilon decays after 0.8979964256286621 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.575221238938052 

*********  Elapsed time: 71.55718207359314 seconds  *************

------Epsilon decays after 0.879370927810669 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.809090909090909 

*********  Elapsed time: 72.44764447212219 seconds  *************

------Epsilon decays after 0.8894631862640381 se

*********  Elapsed time: 103.99995851516724 seconds  *************

------Epsilon decays after 0.9739818572998047 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.56637168141593 

*********  Elapsed time: 104.9369626045227 seconds  *************

------Epsilon decays after 0.9380037784576416 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.705357142857142 

*********  Elapsed time: 105.88096117973328 seconds  *************

------Epsilon decays after 0.9429988861083984 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.711711711711711 

*********  Elapsed time: 106.77546715736389 seconds  *************

------Epsilon decays after 0.8945059776306152 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.382608695652173 

*********  Elapsed time: 107.66370224952698 seconds  *************

------Epsilon decays after 0.8

*********  Elapsed time: 139.96909189224243 seconds  *************

------Epsilon decays after 0.8900036811828613 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.642857142857142 

*********  Elapsed time: 140.85814571380615 seconds  *************

------Epsilon decays after 0.8890538215637207 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.43859649122807 

*********  Elapsed time: 141.75302457809448 seconds  *************

------Epsilon decays after 0.8958756923675537 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.584070796460177 

*********  Elapsed time: 142.6568455696106 seconds  *************

------Epsilon decays after 0.9028241634368896 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.473684210526315 

*********  Elapsed time: 143.5478000640869 seconds  *************

------Epsilon decays after 0.89

...

*********  Elapsed time: 373.09843397140503 seconds  *************

------Epsilon decays after 1.2149651050567627 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.73873873873874 

*********  Elapsed time: 374.37002205848694 seconds  *************

------Epsilon decays after 1.2715880870819092 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.854545454545455 

*********  Elapsed time: 375.71236872673035 seconds  *************

------Epsilon decays after 1.34035062789917 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.625 

*********  Elapsed time: 377.0335614681244 seconds  *************

------Epsilon decays after 1.3201780319213867 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  11.075471698113208 

*********  Elapsed time: 378.41532278060913 seconds  *************

------Epsilon decays after 1.382762193679809

*********  Elapsed time: 416.3096492290497 seconds  *************

------Epsilon decays after 1.0366065502166748 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.382608695652173 

*********  Elapsed time: 417.3240144252777 seconds  *************

------Epsilon decays after 1.0143651962280273 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.747747747747749 

*********  Elapsed time: 418.3383758068085 seconds  *************

------Epsilon decays after 1.0153653621673584 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.935779816513762 

*********  Elapsed time: 419.3541979789734 seconds  *************

------Epsilon decays after 1.0148181915283203 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  11.19626168224299 

*********  Elapsed time: 420.38185715675354 seconds  *************

------Epsilon decays after 1.0276

*********  Elapsed time: 454.52437710762024 seconds  *************

------Epsilon decays after 1.0194056034088135 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.678571428571429 

*********  Elapsed time: 455.5565195083618 seconds  *************

------Epsilon decays after 1.0321424007415771 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  11.019047619047619 

*********  Elapsed time: 456.6005148887634 seconds  *************

------Epsilon decays after 1.0429928302764893 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.73873873873874 

*********  Elapsed time: 457.63816690444946 seconds  *************

------Epsilon decays after 1.0376520156860352 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  11.158878504672897 

*********  Elapsed time: 458.6951570510864 seconds  *************

------Epsilon decays after 1.055

*********  Elapsed time: 495.28036761283875 seconds  *************

------Epsilon decays after 1.156768560409546 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  11.20754716981132 

*********  Elapsed time: 496.45475578308105 seconds  *************

------Epsilon decays after 1.1743881702423096 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.067226890756302 

*********  Elapsed time: 497.66600728034973 seconds  *************

------Epsilon decays after 1.210249900817871 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  11.046296296296296 

*********  Elapsed time: 498.73571014404297 seconds  *************

------Epsilon decays after 1.0706920623779297 seconds since last decay-----

old epsilon: 0.2 

new epsilon: 0.2

Average states alive:  10.801801801801801 

*********  Elapsed time: 499.8703052997589 seconds  *************

------Epsilon decays after 1.132

KeyboardInterrupt: 